In [1]:
# training_and_features.py
import os
import numpy as np
import pandas as pd
import pickle
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
import matplotlib.pyplot as plt

# Load labels
labels_df = pd.read_csv("labels.csv",encoding='latin')
labels_df = labels_df[["ï»¿Image", "Class", "Description"]]

# Encode labels
le = LabelEncoder()
labels_df["label_encoded"] = le.fit_transform(labels_df["Class"])

# Load images
image_dir = "image"
image_data, image_labels, metadata, feature_vectors = [], [], [], []

# Feature extractor model
base_model = MobileNetV2(weights="imagenet", include_top=False, pooling="avg")
feature_model = Model(inputs=base_model.input, outputs=base_model.output)

for filename in os.listdir(image_dir):
    if filename in labels_df["ï»¿Image"].values:
        try:
            path = os.path.join(image_dir, filename)
            img = Image.open(path).convert("RGB").resize((224, 224))
            array = np.array(img) / 255.0
            image_data.append(array)

            # Labels
            label = labels_df.loc[labels_df["ï»¿Image"] == filename, "label_encoded"].values[0]
            image_labels.append(label)

            # Metadata
            cls = labels_df.loc[labels_df["ï»¿Image"] == filename, "Class"].values[0]
            desc = labels_df.loc[labels_df["ï»¿Image"] == filename, "Description"].values[0]
            metadata.append((filename, cls, desc))

            # Feature Vector
            pre_img = preprocess_input(img_to_array(img))
            fv = feature_model.predict(np.expand_dims(pre_img, axis=0), verbose=0)[0]
            feature_vectors.append(fv)

        except Exception as e:
            print(f"❌ Error loading {filename}: {e}")

# Save metadata and vectors
np.save("feature_vectors.npy", np.array(feature_vectors))
with open("metadata.pkl", "wb") as f:
    pickle.dump(metadata, f)
with open("label_encoder.pkl", "wb") as f:
    pickle.dump(le, f)

# Train CNN
X = np.array(image_data)
y = to_categorical(np.array(image_labels))
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(y.shape[1], activation='softmax')
])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=30, batch_size=32, validation_data=(X_val, y_val),
                    callbacks=[EarlyStopping(patience=5, restore_best_weights=True)])
model.save("artifact_model_0.keras")


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

C:\Users\beatb\anaconda3\envs\tf13-py310\lib\site-packages\PIL\Image.py:3570: UserWarning: image file could not be identified because AVIF support not installed
  warnings.warn(message)


❌ Error loading coin37.jpg: cannot identify image file 'image\\coin37.jpg'
Epoch 1/30
5/5 [==============================] - 5s 793ms/step - loss: 3.6126 - accuracy: 0.3088 - val_loss: 1.3723 - val_accuracy: 0.2857
Epoch 2/30
5/5 [==============================] - 4s 703ms/step - loss: 1.3845 - accuracy: 0.3529 - val_loss: 1.3695 - val_accuracy: 0.2000
Epoch 3/30
5/5 [==============================] - 4s 705ms/step - loss: 1.2427 - accuracy: 0.4779 - val_loss: 1.4490 - val_accuracy: 0.4286
Epoch 4/30
5/5 [==============================] - 4s 817ms/step - loss: 1.0223 - accuracy: 0.7059 - val_loss: 0.9421 - val_accuracy: 0.7429
Epoch 5/30
5/5 [==============================] - 5s 981ms/step - loss: 0.8262 - accuracy: 0.7721 - val_loss: 0.7697 - val_accuracy: 0.7143
Epoch 6/30
5/5 [==============================] - 5s 955ms/step - loss: 0.5107 - accuracy: 0.8309 - val_loss: 0.6072 - val_accuracy: 0.7143
Epoch 7/30
5/5 [==============================] - 5s 1s/step - loss: 0.3814 - accurac

In [2]:
import os

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = 'Artifact recognition/axial-glow-457306-g3-e0baa3ca77ed.json'


In [3]:
import os
import numpy as np
import pickle
from PIL import Image
from sklearn.metrics.pairwise import cosine_similarity
import gradio as gr
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, MobileNetV2
from tensorflow.keras.preprocessing.image import img_to_array
from google.cloud import vision
import wikipedia

# Set Google credentials if needed
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = './axial-glow-457306-g3-e0baa3ca77ed.json' # Update this path!

# Load saved data
features = np.load("feature_vectors.npy")
with open("metadata.pkl", "rb") as f:
    metadata = pickle.load(f)
with open("label_encoder.pkl", "rb") as f:
    le = pickle.load(f)
cnn_model = load_model("artifact_model_0.keras")

# MobileNetV2 for feature extraction
base_model = MobileNetV2(weights="imagenet", include_top=False, pooling="avg")
feature_model = Model(inputs=base_model.input, outputs=base_model.output)

# Initialize Google Vision client
vision_client = vision.ImageAnnotatorClient()

# Function to detect object name using Google Vision API
def detect_object_google_vision(image_path):
    with open(image_path, "rb") as image_file:
        content = image_file.read()

    image = vision.Image(content=content)
    response = vision_client.web_detection(image=image)
    web_detection = response.web_detection

    if web_detection.web_entities:
        # Take the top web entity (more artifact/object focused)
        object_name = web_detection.web_entities[0].description
        return object_name
    else:
        return None

# Function to fetch a description from Wikipedia
def fetch_wikipedia_summary(object_name):
    try:
        summary = wikipedia.summary(object_name, sentences=3)
        return summary
    except Exception as e:
        return f"No Wikipedia summary found for {object_name}."

# Main function wrapped for Gradio
def identify_image_gradio(image):
    # Save uploaded image temporarily
    temp_path = "temp_uploaded_image.jpg"
    image.save(temp_path)

    # Step 1: Feature vector extraction
    img = Image.open(temp_path).convert("RGB").resize((224, 224))
    preprocessed = preprocess_input(img_to_array(img).astype("float32"))
    fv_input = feature_model.predict(np.expand_dims(preprocessed, axis=0), verbose=0)[0]

    # Step 2: Cosine similarity
    similarities = cosine_similarity([fv_input], features)[0]
    best_match_idx = np.argmax(similarities)

    if similarities[best_match_idx] >= 0.97:
        filename, class_name, description = metadata[best_match_idx]
        object_detected = "Exact match found"
    else:
        # CNN prediction
        img_array = np.array(img) / 255.0
        pred_class_idx = np.argmax(cnn_model.predict(np.expand_dims(img_array, axis=0), verbose=0))
        class_name = le.inverse_transform([pred_class_idx])[0]

        # Google Vision + Wikipedia
        object_name = detect_object_google_vision(temp_path)
        if object_name:
            description = fetch_wikipedia_summary(object_name)
            object_detected = object_name
        else:
            description = "Object detection failed. No description available."
            object_detected = "Unknown"

    return class_name, object_detected, description

# Gradio Interface
interface = gr.Interface(
    fn=identify_image_gradio,
    inputs=gr.Image(type="pil"),
    outputs=[
        gr.Textbox(label="Predicted Class"),
        gr.Textbox(label="Detected Object Name"),
        gr.Textbox(label="Historical Significance (Wikipedia Description)")
    ],
    title="Artifact Recognition App",
    description="Upload an artifact image. The app identifies it using cosine similarity, CNN, Google Vision API, and Wikipedia."
)

interface.launch(debug=True)


C:\Users\beatb\anaconda3\envs\tf13-py310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.
